# Rxiv-Maker: Automated LaTeX Article Generation

This notebook allows you to run Rxiv-Maker in Google Colab to generate publication-ready LaTeX articles from Markdown manuscripts.

**Features:**
- Convert Markdown to LaTeX with academic formatting
- Generate figures from Python scripts and Mermaid diagrams
- Complete LaTeX compilation with bibliography
- Download generated PDF and source files

**What you'll need:**
- Your manuscript in Markdown format with YAML metadata
- Bibliography file (.bib)
- Figure source files (Python scripts or Mermaid diagrams)

## 🚀 Setup and Installation

First, let's install all the required dependencies.

In [ ]:
# @title Install all dependencies
# Install LaTeX distribution
!apt-get -qq update
!apt-get -qq install texlive texlive-latex-extra texlive-fonts-recommended texlive-fonts-extra texlive-science texlive-bibtex-extra lmodern latexmk ghostscript -y

# Manually install missing styles like siunitx
!tlmgr init-usertree
!tlmgr install siunitx textgreek ifsym

# Make sure binaries are in the PATH
import os
os.environ["PATH"] += ":/usr/bin"


# Install Python dependencies
!pip install matplotlib>=3.7.0 seaborn>=0.12.0 numpy>=1.24.0 pandas>=2.0.0 scipy>=1.10.0 -q -q -q
!pip install Pillow>=9.0.0 pypdf>=3.0.0 PyYAML>=6.0.0 -q -q -q
!pip install ezinput  -q -q -q

# Install Mermaid CLI for diagram generation
!npm install -g @mermaid-js/mermaid-cli

print("✅ All dependencies installed successfully!")

## 💾 Choose between storing the files in current Colab Session or in your Google Drive


In [ ]:
#@title Where do you want to store the files?
import os
from ezinput import EZInput as ezi

gui = ezi()
gui.add_label("Do you want to save the cloned repository and generated files in Google Drive?")
gui.add_label("If not, cloned repo and files will be stored in the current Colab runtime.")
gui.add_check("in_gdrive", description="Store in Google Drive?", remember_value=True, value=False)
gui.add_label("   ")
gui.add_label("Type the name for the manuscript folder to be used.")
gui.add_label("If a name different from the default MANUSCRIPT folder is selected for the first time, make sure to also select 'Create new manuscript folder'.")
gui.add_text_area("man_name", description="Manuscript Name:", remember_value=True, placeholder="MANUSCRIPT", value="MANUSCRIPT")
gui.add_label("   ")
gui.add_label("Select if you are using a new name for the manuscript folder.")
gui.add_check("new_folder", description="Create new manuscript folder", remember_value=True, value=False)
gui.add_label("   ")
gui.add_label("Select 'Continue a previous build' to use previously created files stored in your Google Drive.")
gui.add_label("If not selected a new default folder will be created and possibly overwrite previously created files.")
gui.add_check("continue", description="Continue a previous build?", remember_value=True, value=False)
gui.add_label("   ")
gui.add_label("After selecting the desired options, run the next cell as options are automatically loaded.")
gui.show()

## 📥 Clone Rxiv-Maker Repository

In [ ]:
#@title Clone the Rxiv-Maker repository to access the processing scripts and templates.
import os
import shutil

if gui["in_gdrive"].value:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/rxiv-maker"
    os.environ["MANUSCRIPT_PATH"] = "drive/MyDrive/rxiv-maker" + os.sep + gui["man_name"].value
else:
    PATH = "/content/rxiv-maker"
    os.environ["MANUSCRIPT_PATH"] = "rxiv-maker" + os.sep + gui["man_name"].value

output_path = PATH + os.sep + "output"
output_figures_path = PATH + os.sep + gui["man_name"].value + os.sep + "FIGURES"

if not gui["continue"].value:
  # Clone the repository
  if os.path.exists(PATH):
      shutil.rmtree(PATH)

  !git clone https://github.com/HenriquesLab/rxiv-maker.git {PATH}

  # Create output directory
  !mkdir -p {output_path}
  !mkdir -p {output_figures_path}

  print("✅ Rxiv-Maker repository cloned and set up!")
  print(f"Current directory: {os.getcwd()}")
  #!ls -la

  if gui["new_folder"].value:
    if os.path.exists(PATH + os.sep + gui["man_name"].value):
        shutil.rmtree(PATH + os.sep + gui["man_name"].value)
    shutil.copytree(PATH + os.sep + "MANUSCRIPT", PATH + os.sep + gui["man_name"].value)
  elif gui["man_name"].value == "EXAMPLE_MANUSCRIPT":
    pass # will use the example manuscript folder
  else:
    os.rename(PATH + os.sep + "MANUSCRIPT", PATH + os.sep + gui["man_name"].value)

!pip install {PATH} -q -q -q

## 🔧 Generate Figures

In [ ]:
#@title Generate figures from Python scripts and Mermaid diagrams in the FIGURES directory.
# Check what figure sources we have
print("Figure source files:")
!ls -la {figures_path}
figures_path = PATH + os.sep + gui["man_name"].value + os.sep + "FIGURES"
# Generate figures
print("\n" + "=" * 50)
print("Generating figures...")
print("=" * 50)
generate_path = PATH + os.sep + "src" + os.sep + "py" + os.sep + "commands" + os.sep + "generate_figures.py"
!python3 {generate_path} --figures-dir {figures_path} --output-dir {output_figures_path} --format png

print("\n✅ Figure generation completed!")

## 📄 Generate LaTeX Article

In [ ]:
#@title Convert the Markdown manuscript to LaTeX using the Rxiv-Maker processing pipeline.
print("Generating LaTeX article from Markdown...")
print("=" * 50)

generate_preprint_path = PATH + os.sep + "src" + os.sep + "py" + os.sep + "commands" + os.sep + "generate_preprint.py"
!python3 {generate_preprint_path} --output-dir {output_path}

print("\n✅ LaTeX generation completed!")
print("\nContents of output directory:")
!ls -la {output_path}

## 📚 Copy Supporting Files

In [ ]:
#@title Copy all necessary LaTeX style files, bibliography, and figures to the output directory.
import glob
import shutil
import os

print("Copying supporting files to output directory...")
print("=" * 50)

# Copy LaTeX style files
style_files = glob.glob(PATH + os.sep + "src/tex/style/*")
for file in style_files:
    if os.path.isfile(file):
        shutil.copy2(file, output_path)
        print(f"✅ Copied {os.path.basename(file)}")

# Copy bibliography
bib_path = os.path.join(PATH, gui["man_name"].value, "02_REFERENCES.bib")
if os.path.exists(bib_path):
    shutil.copy2(bib_path, output_path)
    print("✅ Copied bibliography")

# Copy figures directory recursively
figures_src = os.path.join(PATH, gui["man_name"].value, "FIGURES")
figures_dst = os.path.join(output_path, "Figures")

if os.path.exists(figures_src):
    if os.path.exists(figures_dst):
        shutil.rmtree(figures_dst)  # remove if exists to avoid duplication
    shutil.copytree(figures_src, figures_dst)
    print("✅ Copied all figures and subfolders")

print("\n✅ All supporting files copied!")
print("\nFinal output directory contents:")
!ls -la {output_path}
print("\nFigures directory:")
!ls -la {figures_dst}

## 🔨 Compile LaTeX to PDF

In [ ]:
#@title Compile the LaTeX document to generate the final PDF. This process includes multiple passes for proper cross-references and bibliography.
# Change to output directory for compilation
%cd {output_path}
print("Compiling LaTeX to PDF...")
print("This may take a few minutes...")
print("=" * 50)

# First LaTeX pass
print("🔄 First LaTeX pass...")
article_path = gui["man_name"].value + ".tex"
article_path_pdf = gui["man_name"].value + ".pdf"
article_log_path = gui["man_name"].value + ".log"
!pdflatex -interaction=nonstopmode {article_path}

# Bibliography processing
print("\n📚 Processing bibliography...")
!bibtex {article_path[:-4]}

# Second LaTeX pass
print("\n🔄 Second LaTeX pass...")
!pdflatex -interaction=nonstopmode {article_path}

# Third LaTeX pass (for final cross-references)
print("\n🔄 Final LaTeX pass...")
!pdflatex -interaction=nonstopmode {article_path}

print("\n" + "=" * 50)
if os.path.exists(article_path_pdf):
    print("🎉 PDF compilation completed successfully!")
    print(f"📄 Generated PDF: {article_path_pdf} ({os.path.getsize(article_path_pdf)} bytes)")
else:
    print("❌ PDF compilation failed. Check the log file for errors.")
    print("\nLaTeX log (last 50 lines):")
    !tail -50 {article_log_path}
%cd /content/


## 📱 Preview and Download

In [ ]:
#@title View the generated PDF and download all output files.
import base64

from IPython.display import HTML, display

# Check if PDF was generated
pdf_path = output_path + os.sep + gui["man_name"].value + ".pdf"
if os.path.exists(pdf_path):
    print("📄 PDF generated successfully!")

    # Display PDF preview
    with open(pdf_path, "rb") as f:
        pdf_data = f.read()

    pdf_b64 = base64.b64encode(pdf_data).decode("utf-8")
    pdf_display = f'<iframe src="data:application/pdf;base64,{pdf_b64}" width="100%" height="600px"></iframe>'
    display(HTML(pdf_display))

    print("\n" + "=" * 50)
    print("📥 Download your files:")
    print("Run the next cells to download the manuscript files and/or all files as a ZIP.")
    print("=" * 50)

else:
    print("❌ PDF not found. Please check the compilation step above.")
    print("\nAvailable files in output directory:")
    !ls -la {output_path}

In [ ]:
#@title Download Manuscript Files
from google.colab import files
# Download individual files
print("Click to download individual files:")
print("-" * 40)

# Download PDF
if os.path.exists(pdf_path):
    files.download(pdf_path)
    print(f"✅ {article_path_pdf} downloaded")

# Download LaTeX source
if os.path.exists(output_path + os.sep + article_path):
    files.download(output_path + os.sep + article_path)
    print(f"✅ {article_path} downloaded")

print("\n📦 Or download everything as ZIP:")

In [ ]:
#@title Download all files as a ZIP
# Create ZIP archive with all output files
import datetime
import zipfile

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"rxiv_maker_output_{timestamp}.zip"

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    # Add all files from output directory
    for root, dirs, files_list in os.walk(output_path):
        for file in files_list:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, output_path)
            zipf.write(file_path, arcname)
            print(f"📁 Added to ZIP: {arcname}")

print(f"\n✅ Created ZIP archive: {zip_filename}")
print(f"📦 Archive size: {os.path.getsize(zip_filename)} bytes")

# Download the ZIP file
files.download(zip_filename)
print("🎉 ZIP archive downloaded successfully!")

## 🔧 Troubleshooting

In [ ]:
#@title If you encounter issues, run this cell to check common problems.
print("🔍 TROUBLESHOOTING CHECKLIST")
print("=" * 50)
files_path = PATH + os.sep + gui["man_name"].value
# Check required files
required_files = [PATH + os.sep + gui["man_name"].value + os.sep + "01_MAIN.md", PATH + os.sep + gui["man_name"].value + os.sep + "03_REFERENCES.bib"]
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file} found")
    else:
        print(f"❌ {file} missing")

# Check Python scripts
python_scripts = [
    PATH + os.sep + "src/py/commands/generate_preprint.py",
    PATH + os.sep + "src/py/commands/generate_figures.py",
]
for script in python_scripts:
    if os.path.exists(script):
        print(f"✅ {script} found")
    else:
        print(f"❌ {script} missing")

# Check LaTeX installation
import subprocess

try:
    result = subprocess.run(["pdflatex", "--version"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ pdflatex installed")
    else:
        print("❌ pdflatex not working")
except FileNotFoundError:
    print("❌ pdflatex not found")

# Check output directory
if os.path.exists(PATH + os.sep + "output"):
    print("✅ Output directory exists")
    files_in_output = os.listdir(PATH + os.sep + "output")
    print(f"📁 Files in output: {len(files_in_output)}")
    if article_path in files_in_output:
        print(f"✅ {article_path} generated")
    else:
        print(f"❌ {article_path} not found")
    if article_path_pdf in files_in_output:
        print(f"✅ {article_path_pdf} generated")
    else:
        print(f"❌ {article_path_pdf} not found")
else:
    print("❌ Output directory missing")

print("\n" + "=" * 50)
print("If you see any ❌ errors above, please:")
print("1. Re-run the setup cells")
print("2. Check that your input files are properly formatted")
print("3. Look at the LaTeX log for compilation errors")

# Show recent LaTeX log if available
if os.path.exists(f"output/{gui['man_name'].value}.log"):
    print("\n📄 Recent LaTeX log (last 20 lines):")
    !tail -20 output/{gui['man_name'].value}.log

## 📖 Next Steps

**Congratulations!** You've successfully used Rxiv-Maker to generate a publication-ready LaTeX article.

### What you can do next:

1. **Customize your manuscript**: Edit the YAML frontmatter in your Markdown file to change title, authors, affiliations, etc.

2. **Add more figures**: Create Python scripts or Mermaid diagrams in the FIGURES directory

3. **Modify the LaTeX template**: The template is in `src/tex/template.tex`

4. **Use locally**: Clone the repository to your local machine for offline use

### Learn more:
- [Rxiv-Maker Documentation](https://github.com/HenriquesLab/rxiv-maker)
- [Markdown Guide](https://www.markdownguide.org/)
- [LaTeX Documentation](https://www.latex-project.org/help/documentation/)

### Need help?
- Open an issue on [GitHub](https://github.com/HenriquesLab/rxiv-maker/issues)
- Check the troubleshooting section above

---

*This notebook was created for the Rxiv-Maker project by the HenriquesLab.*